In [ ]:
%matplotlib qt5

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle
#plt.ion()

# Event Filtering
We have created some very simple examples that simply report information upon events, but nothing more useful. Let us now create something useful, like a plotter. A plotter will need to record the locations of mouse clicks and draw lines between the clicks. A plotter should also respond to other events in order to change the line color or somesuch.

Previously, we created a callable class where the object itself would act as the callback to multiple event types. The call method could then decide what action to take based upon the information in the event object. Another approach is to create a class with multiple methods that are individually connected to different event types. Both are perfectly valid approaches, each with design pros and cons.

The `LineMaker` class will take a line object from `plt.plot()`, and record its x/y data as Python lists. The data in the line object itself is stored as a NumPy array, which is fixed in size. Then, `LineMaker` will hook up its `on_button()` method to mouse button presses, and its `on_key()` method to key press events.

In [ ]:
class LineMaker:
    def __init__(self, ln):
        # stash the current data
        self.xdata = list(ln.get_xdata())
        self.ydata = list(ln.get_ydata())
        # stash the Line2D artist
        self.ln = ln
        self.color_cyle = cycle(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                                 '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                                 '#bcbd22', '#17becf'])
        self.button_cid = ln.figure.canvas.mpl_connect('button_press_event',
                                                       self.on_button)
        self.key_cid = ln.figure.canvas.mpl_connect('key_press_event',
                                                    self.on_key)

    def on_button(self, event):
        # only consider events from the lines Axes
        if event.inaxes is not self.ln.axes:
            return

        # if not the left mouse button or a modifier key
        # is held down, bail
        if event.button != 1 or event.key is not None:
            print('key+button: {!r}+{!r}'.format(event.key, event.button))
            return

        # get the event location in data-space
        self.xdata.append(event.xdata)
        self.ydata.append(event.ydata)

        # update the artist data
        self.ln.set_data(self.xdata, self.ydata)

        # ask the GUI to re-draw the next time it can
        self.ln.figure.canvas.draw_idle()

    def on_key(self, event):
        # This is _super_ useful for debugging!
        # print(event.key)

        # if the key is c (any case)
        if event.key.lower() == 'c':
            # change the color
            self.ln.set_color(next(self.color_cyle))

            # ask the GUI to re-draw the next time it can
            self.ln.figure.canvas.draw_idle()

If anything but the left mouse button is pressed and/or any key is also being held down, nothing interesting happens. If the mouse event happens outside the plot area, nothing interesting happens as well. But regular left-button presses will cause `LineMaker` to add that data coordinate to its lists of x and y coordinates, and update the Line artist object.

If the 'c' key is pressed, then the line will change color.

In [ ]:
fig, ax = plt.subplots()
ln, = ax.plot([], [], '-o')
line_maker = LineMaker(ln)

# EXERCISE
 - modify to remove the closest point on button == 3 or key == 'shift'
 - change the line width for keys [1-9]
 - clear the line when event.key == 'escape'